In [1]:
import yaml
import os
import re 
import random
from langchain_anthropic import ChatAnthropic

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
anthropic_key = config["anthropic_key"]
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

In [2]:
from wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page = get_random_page(1)[0]
start_page = "United_States_at_the_2008_Summer_Olympics"
final_page = "United States"
final_page_content = get_page_content(final_page)
print(start_page)

['Scott Spann (swimmer)', "Athletics at the 2008 Summer Olympics – Men's decathlon", 'Chonnapas Premwaew', 'Amel Mekić', 'Rebecca Soumeru', "2008 CONCACAF Men's Pre-Olympic Tournament", 'Nancy Rios', 'Brian Olson', 'super heavyweight', "Cycling at the 2008 Summer Olympics – Women's cross-country", 'Jason Donald (baseball)', 'Masahiro Araki', 'Mike Friedman', 'Lindsey Berg', 'Brian Barden', 'Hyleas Fountain', "Diving at the 2008 Summer Olympics – Women's synchronized 10 metre platform", 'Li Jiawei', 'Peter Varellas', 'Brad Walker (athlete)', 'Sailing at the 2008 Summer Olympics – Finn class', 'Breaux Greer', 'Lovieanne Jung', 'LaShauntea Moore', "Cycling at the 2008 Summer Olympics – Women's sprint", 'Zhu Jun (fencer)', 'Lindsay Meyer', 'Ai Sugiyama', "Canoeing at the 2008 Summer Olympics – Women's slalom K-1", "Wrestling at the 2008 Summer Olympics – Men's freestyle 66 kg", 'Ogonna Nnamani', 'Carl Lewis', 'Vancouver, British Columbia', 'Georgia Gould (cyclist)', 'Carrie Howe', 'Chih-Sh

In [4]:
end_summary = chain.invoke(
        {
            "page_content": final_page_content
        }
    )
model_output = end_summary.content
pattern = r'<summary>(.*?)</summary>'
matches = re.findall(pattern, model_output)
end_content = matches[0]

In [5]:
start_links = get_page_links(start_page)
print(start_links)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

['Scott Spann (swimmer)', "Athletics at the 2008 Summer Olympics – Men's decathlon", 'Chonnapas Premwaew', 'Amel Mekić', 'Rebecca Soumeru', "2008 CONCACAF Men's Pre-Olympic Tournament", 'Nancy Rios', 'Brian Olson', 'super heavyweight', "Cycling at the 2008 Summer Olympics – Women's cross-country", 'Jason Donald (baseball)', 'Masahiro Araki', 'Mike Friedman', 'Lindsey Berg', 'Brian Barden', 'Hyleas Fountain', "Diving at the 2008 Summer Olympics – Women's synchronized 10 metre platform", 'Li Jiawei', 'Peter Varellas', 'Brad Walker (athlete)', 'Sailing at the 2008 Summer Olympics – Finn class', 'Breaux Greer', 'Lovieanne Jung', 'LaShauntea Moore', "Cycling at the 2008 Summer Olympics – Women's sprint", 'Zhu Jun (fencer)', 'Lindsay Meyer', 'Ai Sugiyama', "Canoeing at the 2008 Summer Olympics – Women's slalom K-1", "Wrestling at the 2008 Summer Olympics – Men's freestyle 66 kg", 'Ogonna Nnamani', 'Carl Lewis', 'Vancouver, British Columbia', 'Georgia Gould (cyclist)', 'Carrie Howe', 'Chih-Sh

In [6]:
from matching_functions import find_closest_documents
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

<All keys matched successfully>


In [7]:
current_page = start_page
current_links = valid_links
forbidden_links = [start_page]
for i in range(15):
    print(current_links)
    closest_links = find_closest_documents(model, final_page, current_links)
    print(closest_links)
    broad_links = broad_links_chain.invoke({
        "link_list": current_links,
        "end_page": final_page,
        "end_page_summary": end_content
    })
    model_output = broad_links.content
    pattern = r'<output>(.*?)</output>'
    matches = re.findall(pattern, model_output)
    links = [link.strip() for link in matches[0].split(",")]
    current_links = closest_links + links
    explain_links = explain_links_chain.invoke({
        "link_list": current_links,
        "end_page": final_page,
        "end_page_summary": end_content
    })
    reasonings = explain_links.content
    crawler_text = crawler_chain.invoke(
        {
            "current_page": current_page,
            "current_links": current_links,
            "reasonings": reasonings,
            "end_page": final_page,
            "end_page_content": end_content
        }
    )
    model_output = crawler_text.content
    print(model_output)
    pattern = r'<output>(.*?)</output>'
    matches = re.findall(pattern, model_output)
    current_page = matches[0]
    if current_page == "Random":
        current_page = random.choice(current_links)
    if current_page != final_page:
        forbidden_links.append(current_page)
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(start_links)
        current_links = [link for link in start_links if checked_links[link] and link not in forbidden_links]
    else:
        print(f"Page reached in {i} iterations!")
        break

['Scott Spann (swimmer)', "Athletics at the 2008 Summer Olympics – Men's decathlon", 'Chonnapas Premwaew', 'Amel Mekić', 'Rebecca Soumeru', "2008 CONCACAF Men's Pre-Olympic Tournament", 'Nancy Rios', 'Brian Olson', "Cycling at the 2008 Summer Olympics – Women's cross-country", 'Jason Donald (baseball)', 'Masahiro Araki', 'Mike Friedman', 'Lindsey Berg', 'Brian Barden', 'Hyleas Fountain', "Diving at the 2008 Summer Olympics – Women's synchronized 10 metre platform", 'Li Jiawei', 'Peter Varellas', 'Brad Walker (athlete)', 'Sailing at the 2008 Summer Olympics – Finn class', 'Breaux Greer', 'Lovieanne Jung', 'LaShauntea Moore', "Cycling at the 2008 Summer Olympics – Women's sprint", 'Zhu Jun (fencer)', 'Lindsay Meyer', 'Ai Sugiyama', "Canoeing at the 2008 Summer Olympics – Women's slalom K-1", "Wrestling at the 2008 Summer Olympics – Men's freestyle 66 kg", 'Ogonna Nnamani', 'Carl Lewis', 'Vancouver, British Columbia', 'Georgia Gould (cyclist)', 'Carrie Howe', 'Chih-Sheng Lin', 'Ilario Di 